In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
email = 'matheus.maranhao@hotmail.com'
pw ='mgm06121995'

In [3]:
#Log-in
browser = webdriver.Firefox()
browser.get('https://auth.reclameaqui.com.br/auth/realms/reclameaqui/protocol/openid-connect/auth?client_id=sitepublico&redirect_uri=https%3A%2F%2Fwww.reclameaqui.com.br%2Fempresa%2Fcielo%2Flista-reclamacoes%2F&state=6a862032-3926-4c2f-9600-b35e0be25bdd&response_mode=fragment&response_type=code&scope=openid&nonce=e8b04d76-d336-4f46-8cf9-fcecca1b6618')

time.sleep(3)

elem = browser.find_element_by_id('username')
elem.send_keys(email)


elem = browser.find_element_by_id('password')
elem.send_keys(pw)

browser.find_element_by_id('kc-login').click()

time.sleep(20)

/tmp/ipykernel_5618/334499702.py:7: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = browser.find_element_by_id('username')
/tmp/ipykernel_5618/334499702.py:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = browser.find_element_by_id('password')
/tmp/ipykernel_5618/334499702.py:14: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  browser.find_element_by_id('kc-login').click()


In [4]:
#Get list page HTML
source = []

from tqdm import tqdm
for i in tqdm(range(1,20)):
    url = 'https://www.reclameaqui.com.br/empresa/cielo/lista-reclamacoes/?pagina='+str(i+1)
    browser.get(url)
    source.append(browser.page_source)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:55<00:00,  2.91s/it]


In [5]:
#Get links href
links = []
for i in tqdm(source):
    soup = BeautifulSoup(i)
    l = soup.find_all('div', {'class':'sc-1pe7b5t-0 bJdtis'})
    for x in l:
        links.append(x.a['href'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 24.54it/s]


In [6]:
#Get source
s = []

for i in tqdm(links):
    url = 'https://www.reclameaqui.com.br'+i
    browser.get(url)
    s.append(browser.page_source)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [08:20<00:00,  2.63s/it]


In [7]:
pergunta = []
d_perg = []
local = []
assunto = []
d_resp = []
resp = []
status = []
for i in tqdm(s):
    soup = BeautifulSoup(i)

    #Pergunta
    l = soup.find('h1', {'class':'lzlu7c-3 berwWw'})
    pergunta.append(l.text)

    #Data
    l = soup.find('span', {'data-testid':"complaint-creation-date"})
    d_perg.append(l.text)

    # Local
    l = soup.find('span', {'data-testid':"complaint-location"})
    local.append(l.text)

    #Assunto
    l = soup.find_all('ul',{'class':'sc-1dmxdqs-0 kgaQNj'})
    l = l[1].find_all('li')
    n=[]
    for i in range(len(l)-1):
        n.append(l[i+1].text)
    assunto.append(n)
    try:
        #Data da Resposta
        l = soup.find('span', {'class':"sc-1o3atjt-3 ipwWvs"})
        d_resp.append(l.text)

        #Resposta
        l = soup.find('p', {'class':"sc-1o3atjt-4 JkSWX"})
        resp.append(l.text)
    except:
        d_resp.append(np.nan)
        resp.append(np.nan)

    #Status
    l = soup.find('div', {'data-testid':"complaint-status"})
    status.append(l.text)
    
d={}

d['d_perg']=d_perg
d['pergunta']=pergunta
d['assunto']=assunto
d['local']=local
d['d_resp']=d_resp
d['resp']=resp
d['status']=status

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 190/190 [00:05<00:00, 35.03it/s]


In [9]:
df = pd.DataFrame(d)

In [10]:
df.to_csv('reclame_aqui.csv')